In [2]:
from math import sqrt
import pandas as pd
import numpy as np
from numpy import nan
from pandas import Series,DataFrame

In [3]:
df=pd.read_csv('sample.csv',encoding='shift-jis')

In [35]:
df

,USER,PRODUCT,CATEGOLY
0,A,a,ズボン
1,A,c,ズボン
2,A,e,ズボン
3,B,a,ズボン
4,B,d,ズボン
5,B,e,ズボン
6,C,b,ズボン
7,C,e,ズボン
8,C,f,ズボン
9,D,a,シャツ


In [41]:
y=df['USER'].unique()
y[0]

'A'

In [38]:
z=df['PRODUCT'].unique()
z

array(['a', 'c', 'e', 'd', 'b', 'f'], dtype=object)

In [105]:
arr=np.array(['a', 'c','f'], dtype=object)
arr

array(['a', 'c', 'f'], dtype=object)

In [100]:
x1

array(['a', 'c', 'e'], dtype=object)

In [104]:
x2

3    a
4    d
5    e
Name: PRODUCT, dtype: object

In [101]:
for x in y[0]:
    x1=df[df['USER']==x]['PRODUCT'].unique()

In [107]:
#同じ長さじゃなきゃいけない問題
np.vstack([arr, x1])

array([['a', 'c', 'f'],
       ['a', 'c', 'e']], dtype=object)

In [69]:
for x in y[1]:
    x2=df[df['USER']==x]['PRODUCT']
    
x2

3    a
4    d
5    e
Name: PRODUCT, dtype: object

In [72]:
arr=np.vstack([x1, x2])
arr

array([['a', 'c', 'e'],
       ['a', 'd', 'e']], dtype=object)

In [63]:
len(np.intersect1d(x1, x2))/len(np.union1d(x1, x2))

0.5

In [ ]:
def jaccard(prefs,v1, v2):


In [11]:
# Returns a distance-based similarity score for person1 and person2
def sim_distance(prefs,person1,person2):
  # Get the list of shared_items
  si={}
  for item in prefs[person1]: 
    if item in prefs[person2]: si[item]=1

  # if they have no ratings in common, return 0
  if len(si)==0: return 0

  # Add up the squares of all the differences
  sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2) 
                      for item in prefs[person1] if item in prefs[person2]])

  return 1/(1+sum_of_squares)

# Returns the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs,p1,p2):
  # Get the list of mutually rated items
  si={}
  for item in prefs[p1]: 
    if item in prefs[p2]: si[item]=1

  # if they are no ratings in common, return 0
  if len(si)==0: return 0

  # Sum calculations
  n=len(si)
  
  # Sums of all the preferences
  sum1=sum([prefs[p1][it] for it in si])
  sum2=sum([prefs[p2][it] for it in si])
  
  # Sums of the squares
  sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
  sum2Sq=sum([pow(prefs[p2][it],2) for it in si])	
  
  # Sum of the products
  pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
  
  # Calculate r (Pearson score)
  num=pSum-(sum1*sum2/n)
  den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
  if den==0: return 0

  r=num/den

  return r


def jaccard(prefs,v1, v2):
    numerator = sum([c in prefs[v2] for c in prefs[v1]])
    denominator = len(prefs[v1]) + len(prefs[v2]) - numerator
    return float(numerator) / denominator if denominator != 0 else 0


# Returns the best matches for person from the prefs dictionary. 
# Number of results and similarity function are optional params.
def topMatches(prefs,person,n=5,similarity=jaccard):
  scores=[(similarity(prefs,person,other),other) 
                  for other in prefs if other!=person]
  scores.sort()
  scores.reverse()
  return scores[0:n]

# Gets recommendations for a person by using a weighted average
# of every other user's rankings
def getRecommendations(prefs,person,similarity=jaccard):
  totals={}
  simSums={}
  for other in prefs:
    # don't compare me to myself
    if other==person: continue
    sim=similarity(prefs,person,other)

    # ignore scores of zero or lower
    if sim<=0: continue
    for item in prefs[other]:
	    
      # only score movies I haven't seen yet
      if item not in prefs[person] or prefs[person][item]==0:
        # Similarity * Score
        totals.setdefault(item,0)
        totals[item]+=prefs[other][item]*sim
        # Sum of similarities
        simSums.setdefault(item,0)
        simSums[item]+=sim

  # Create the normalized list
  rankings=[(total/simSums[item],item) for item,total in totals.items()]

  # Return the sorted list
  rankings.sort()
  rankings.reverse()
  return rankings

def transformPrefs(prefs):
  result={}
  for person in prefs:
    for item in prefs[person]:
      result.setdefault(item,{})
      
      # Flip item and person
      result[item][person]=prefs[person][item]
  return result


def calculateSimilarItems(prefs,n=10):
  # Create a dictionary of items showing which other items they
  # are most similar to.
  result={}
  # Invert the preference matrix to be item-centric
  itemPrefs=transformPrefs(prefs)
  c=0
  for item in itemPrefs:
    # Status updates for large datasets
    c+=1
    if c%100==0: print("%d / %d" % (c,len(itemPrefs)))
    # Find the most similar items to this one
    scores=topMatches(itemPrefs,item,n=n,similarity=jaccard)
    result[item]=scores
  return result

def getRecommendedItems(prefs,itemMatch,user):
  userRatings=prefs[user]
  scores={}
  totalSim={}
  # Loop over items rated by this user
  for (item,rating) in userRatings.items( ):

    # Loop over items similar to this one
    for (similarity,item2) in itemMatch[item]:

      # Ignore if this user has already rated this item
      if item2 in userRatings: continue
      # Weighted sum of rating times similarity
      scores.setdefault(item2,0)
      scores[item2]+=similarity*rating
      # Sum of all the similarities
      totalSim.setdefault(item2,0)
      totalSim[item2]+=similarity

  # Divide each total score by total weighting to get an average
  rankings=[(score/totalSim[item],item) for item,score in scores.items( )]

  # Return the rankings from highest to lowest
  rankings.sort( )
  rankings.reverse( )
  return rankings

def loadMovieLens(path='/data/movielens'):
  # Get movie titles
  movies={}
  for line in open(path+'/u.item'):
    (id,title)=line.split('|')[0:2]
    movies[id]=title
  
  # Load data
  prefs={}
  for line in open(path+'/u.data'):
    (user,movieid,rating,ts)=line.split('\t')
    prefs.setdefault(user,{})
    prefs[user][movies[movieid]]=float(rating)
  return prefs

In [ ]:
df4=DataFrame()
for i in range(len(x)):
    di={}
    df1=df[df['CATEGORY']==x[i]]
    df2=pd.DataFrame(index=df1['ACCT_ID'].unique(),columns=df1['PRODUCT'].unique())

    for j in range(len(df1['ACCT_ID'])):
        df2.ix[df1.iloc[j,0],df1.iloc[j,1]]=1
        for j in df2.index:
            di[j]=dict(df2.ix[j,(df2.ix[j] >= 1)])
    df3=DataFrame(calculateSimilarItems(di,n=5))
    df4=pd.concat([df4,df3])

In [ ]:
df5=df4.unstack()
df5=df5.dropna()

In [ ]:
df6=pd.DataFrame(index=range(len(df5)),columns=['PRODUCT_A','PRODUCT_B','sim'])
for n in range(len(df5)):
    df6.iloc[n,0]=df5.index[n][0]
    df6.iloc[n,1]=df5.iloc[n][1]
    df6.iloc[n,2]=df5.iloc[n][0]

In [ ]:
df6.to_csv('result_product.csv',encoding='shift-jis')